In [35]:
import nflgame
import mirofeatures as mf
import pandas as pd
import numpy as np
from sklearn import linear_model

In [36]:
ftrs, isPass = mf.getFeatures(2010,2013)

In [37]:
df_ftrs = pd.DataFrame(ftrs)
df_isPass = pd.DataFrame(isPass)
ftrs = np.array(ftrs)
isPass =np.array(isPass)
df_ftrs.head(10)

,down,opponent,position,shotgun,team,time,togo
0,1,MIN,77,1,NO,1796.0,10
1,1,MIN,49,0,NO,1761.0,10
2,1,MIN,37,1,NO,1721.0,10
3,2,MIN,37,0,NO,1716.0,10
4,3,MIN,29,1,NO,1684.0,2
5,1,NO,80,0,MIN,1676.0,10
6,2,NO,77,0,MIN,1636.0,7
7,3,NO,72,0,MIN,1598.0,2
8,1,MIN,73,1,NO,1553.0,10
9,2,MIN,75,0,NO,1514.0,12


## Using a simple linear classifier

To start, we have to drop the categorical features because they don't play well with linear classification. 

In [46]:
X = df_ftrs.drop(['opponent', 'team'], axis = 1)

In [38]:
np.random.seed(12346)
mask = np.full(len(ftrs), False)
mask[:len(ftrs)*3/4] = True
np.random.shuffle(mask)

In [47]:
clf = linear_model.SGDClassifier(tol=.000001)
clf.fit(X[mask],isPass[mask])

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=1e-06, verbose=0, warm_start=False)

In [52]:
print("Success rate in predicting pass (for training data) is %f%%" % (100*clf.score(X[mask], isPass[mask])))
print("Success rate in predicting pass (for test data) is %f%%" % (100*clf.score(X[np.invert(mask)], isPass[np.invert(mask)])))

Success rate in predicting pass (for training data) is 56.932857%
Success rate in predicting pass (for test data) is 57.225550%


In [56]:
np.sum(isPass)/float(len(isPass))

0.56593640994755201

Big surprise! Our naive classifier is only barely better than just always guessing "pass."

In [61]:
naive = [True for i in range(len(isPass))]
for i in range(len(isPass)):
    if df_ftrs['togo'][i] <= 3:
        naive[i] = False

In [63]:
print(np.sum(naive==isPass)/float(len(isPass)))

0.580901235497


More bad news. If YTG < 3 and you guess run and otherwise guess pass, outperforms the classifier.